
# 📓 Бленд сабмитов (ipynb)
Этот блокнот помогает блендить сабмиты (CSV) разными способами: **mean**, **weighted**, **median**, **rank**, **logit**, **geometric**.  
Подходит для соревнований (Kaggle/ODS и т.п.) с одинаковыми `id` и колонкой с предсказаниями.

**Что умеет блокнот:**
- Поиск всех `.csv` в папке/по шаблону (можно и список файлов).
- Проверки на совпадение ID, дубликаты, пропуски.
- Несколько стратегий бленда + комбинации.
- Сохранение результата в `blend.csv` (настраивается).
- Отчёт по корреляциям между сабмитами и важности весов.


## ⚙️ Настройки

In [ ]:

# === Пользовательские настройки ===
# Папка или шаблон для поиска файлов (можно список путей). Примеры:
# PATTERN = "submissions/*.csv"
# PATTERN = ["subs/sub1.csv", "subs/sub2.csv", "subs/sub3.csv"]
PATTERN = "submissions/*.csv"

# Имя колонки с ID и с предсказаниями
ID_COLUMN = "id"
TARGET_COLUMN = "prediction"

# Тип бленда: 'mean' | 'weighted' | 'median' | 'rank' | 'logit' | 'geometric'
BLEND_TYPE = "mean"

# Веса для 'weighted' (если None — попытаемся рассчитать по валидации, иначе используем фиксированные)
WEIGHTS = None  # например: [0.5, 0.3, 0.2]

# Файл с out-of-fold/валидационными таргетами (опционально, чтобы подобрать веса)
# Формат: CSV с колонками ID_COLUMN и 'y' (истинная цель). Если нет — веса задать вручную.
VALIDATION_FILE = None  # например: "oof.csv"

# Папка/имя файла для результата
OUTPUT_FILE = "blend.csv"

# Режим нормализации предиктов перед блендом: None | 'minmax' | 'zscore'
NORMALIZATION = None

# Нужно ли логировать промежуточные результаты
VERBOSE = True


## 📦 Импорт и функции ввода-вывода

In [ ]:

import os
import glob
import numpy as np
import pandas as pd

def find_files(pattern):
    if isinstance(pattern, (list, tuple)):
        files = []
        for p in pattern:
            files.extend(glob.glob(p))
        return sorted(files)
    else:
        return sorted(glob.glob(pattern))

def read_submissions(files, id_col, target_col):
    dfs = []
    names = []
    for f in files:
        df = pd.read_csv(f)
        if id_col not in df.columns:
            raise ValueError(f"{f}: нет колонки ID '{id_col}'")
        if target_col not in df.columns:
            # если колонка предсказаний не найдена — попробуем взять последнюю числовую
            num_cols = [c for c in df.columns if c != id_col and pd.api.types.is_numeric_dtype(df[c])]
            if len(num_cols) == 0:
                raise ValueError(f"{f}: не найдено числовой колонки предсказаний и нет '{target_col}'")
            if VERBOSE:
                print(f"[i] {f}: TARGET_COLUMN '{target_col}' не найден, берём {num_cols[-1]!r}")
            pred_col = num_cols[-1]
        else:
            pred_col = target_col
        # оставляем только нужные колонки
        df = df[[id_col, pred_col]].copy()
        df.rename(columns={pred_col: os.path.splitext(os.path.basename(f))[0]}, inplace=True)
        dfs.append(df)
        names.append(os.path.splitext(os.path.basename(f))[0])
    # последовательные merge'и по ID
    base = dfs[0]
    for d in dfs[1:]:
        base = base.merge(d, on=id_col, how="inner", validate="one_to_one")
    return base, names


## ✅ Проверки и предобработка

In [ ]:

def check_and_prepare(df, id_col):
    # Проверка на дубликаты ID
    dup = df[id_col].duplicated().sum()
    if dup > 0:
        raise ValueError(f"В данных есть дубликаты ID: {dup}")
    # Пропуски
    if df.isna().any().any():
        if VERBOSE:
            na_cnt = df.isna().sum().to_dict()
            print('[!] Обнаружены пропуски:', na_cnt)
        df = df.dropna().copy()
        if VERBOSE:
            print('[i] Пропуски удалены. Осталось строк:', len(df))
    return df

def normalize_columns(df, cols, mode=None):
    if mode is None:
        return df[cols].to_numpy()
    X = df[cols].to_numpy().astype(float)
    if mode == "minmax":
        mn = X.min(axis=0, keepdims=True)
        mx = X.max(axis=0, keepdims=True)
        div = np.where(mx - mn == 0, 1.0, mx - mn)
        return (X - mn) / div
    elif mode == "zscore":
        m = X.mean(axis=0, keepdims=True)
        s = X.std(axis=0, ddof=0, keepdims=True)
        s = np.where(s == 0, 1.0, s)
        return (X - m) / s
    else:
        raise ValueError("NORMALIZATION должен быть None | 'minmax' | 'zscore'")


## 🧪 Методы бленда

In [ ]:

from scipy.stats import rankdata

def blend_mean(X):
    return X.mean(axis=1)

def blend_median(X):
    return np.median(X, axis=1)

def blend_weighted(X, weights):
    w = np.array(weights, dtype=float)
    if w.ndim != 1 or w.shape[0] != X.shape[1]:
        raise ValueError("Длина WEIGHTS должна совпадать с числом файлов.")
    w = w / w.sum()
    return (X * w).sum(axis=1)

def blend_rank(X):
    # ранжируем каждый столбец и усредняем ранги
    R = np.column_stack([rankdata(X[:, j]) for j in range(X.shape[1])])
    return R.mean(axis=1)

def to_logit(p, eps=1e-9):
    p = np.clip(p, eps, 1 - eps)
    return np.log(p / (1 - p))

def from_logit(z):
    return 1.0 / (1.0 + np.exp(-z))

def blend_logit(X):
    # предполагаем вероятности в [0,1]
    Z = np.column_stack([to_logit(X[:, j]) for j in range(X.shape[1])])
    z = Z.mean(axis=1)
    return from_logit(z)

def blend_geometric(X, eps=1e-15):
    Xc = np.clip(X, eps, None)
    return np.exp(np.log(Xc).mean(axis=1))


## 🎯 Подбор весов по валидации (опционально)

In [ ]:

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, roc_auc_score

def guess_task_type(y):
    # если y бинарная — классификация, иначе регрессия
    unique = np.unique(y)
    if np.all(np.isin(unique, [0, 1])) and len(unique) <= 2:
        return "binary"
    return "regression"

def fit_weights_with_validation(val_file, id_col, df_all, target_col_name="y"):
    val = pd.read_csv(val_file)
    if id_col not in val.columns or target_col_name not in val.columns:
        raise ValueError(f"{val_file}: нужны колонки '{id_col}' и '{target_col_name}'")
    data = df_all.merge(val[[id_col, target_col_name]], on=id_col, how="inner")
    X = data.drop(columns=[id_col, target_col_name]).to_numpy()
    y = data[target_col_name].to_numpy()
    task = guess_task_type(y)
    if VERBOSE:
        print(f"[i] Найдено {len(data)} пересечений по валидации. Тип задачи: {task}.")
    # Ridge для устойчивости (веса >= 0 можно позже обрезать, но обычно не требуется)
    model = Ridge(alpha=1e-6, fit_intercept=False, positive=True)
    model.fit(X, y)
    w = model.coef_
    if w.sum() == 0:
        w = np.ones_like(w) / len(w)
    else:
        w = w / w.sum()
    # оценка качества
    y_pred = X.dot(w)
    if task == "binary":
        try:
            score = roc_auc_score(y, y_pred)
            metric = "AUC"
        except Exception:
            score = -mean_squared_error(y, y_pred)  # fallback
            metric = "-MSE"
    else:
        score = -mean_squared_error(y, y_pred)
        metric = "-MSE"
    if VERBOSE:
        print(f"[i] Подобранные веса (сумма=1): {np.round(w, 4)} | вал. метрика {metric}: {score:.6f}")
    return w


## 📈 Диагностика: корреляции предсказаний

In [ ]:

def correlation_report(df, id_col):
    preds = df.drop(columns=[id_col])
    corr = preds.corr()
    display(corr)
    return corr


## ▶️ Запуск бленда

In [ ]:

# 1) найти и прочитать файлы
files = find_files(PATTERN)
if len(files) == 0:
    raise FileNotFoundError("Не найдено CSV по заданному PATTERN.")
if VERBOSE:
    print("Найдены файлы:")
    for f in files:
        print(" -", f)

df_all, names = read_submissions(files, ID_COLUMN, TARGET_COLUMN)
df_all = check_and_prepare(df_all, ID_COLUMN)

if VERBOSE:
    print(f"[i] Размер объединённой таблицы: {df_all.shape}")
    print(f"[i] Сабмиты: {names}")

# 2) диагностический отчёт
corr = correlation_report(df_all, ID_COLUMN)

# 3) подготовка массивов
cols = [c for c in df_all.columns if c != ID_COLUMN]
X = normalize_columns(df_all, cols, NORMALIZATION)

# 4) бленд
if BLEND_TYPE == "mean":
    blended = blend_mean(X)
elif BLEND_TYPE == "median":
    blended = blend_median(X)
elif BLEND_TYPE == "weighted":
    if WEIGHTS is None:
        if VALIDATION_FILE is None:
            raise ValueError("Для 'weighted' либо задайте WEIGHTS, либо укажите VALIDATION_FILE для автоподбора.")
        WEIGHTS = fit_weights_with_validation(VALIDATION_FILE, ID_COLUMN, df_all, target_col_name="y")
    blended = blend_weighted(X, WEIGHTS)
elif BLEND_TYPE == "rank":
    blended = blend_rank(X)
elif BLEND_TYPE == "logit":
    blended = blend_logit(X)
elif BLEND_TYPE == "geometric":
    blended = blend_geometric(X)
else:
    raise ValueError("Неизвестный BLEND_TYPE. Используйте: 'mean' | 'weighted' | 'median' | 'rank' | 'logit' | 'geometric'")

# 5) сохранить
out = df_all[[ID_COLUMN]].copy()
out[TARGET_COLUMN] = blended
out.to_csv(OUTPUT_FILE, index=False)
if VERBOSE:
    print(f"✅ Готово! Результат сохранён в {OUTPUT_FILE}")



## 💡 Советы
- **Rank**-бленд устойчив к разбросу масштабов и часто помогает при метриках на сортировке (AUC/MAP/NDCG).
- **Logit**-бленд используйте только для вероятностей в [0,1].
- **Geometric** подходит для вероятностей и частично напоминает log-усреднение.
- Если сабмиты очень похожи (высокая корреляция), выигрыш от бленда будет маленьким — попробуйте добавить более разные модели.
- Для подбора весов по валидации положите `oof.csv` с колонками `id, y` и поставьте `VALIDATION_FILE = "oof.csv"`.
- Если у разных файлов разные имена колонок предсказаний — блокнот сам возьмёт последнюю числовую колонку, если `TARGET_COLUMN` не найден.
